# MVA - Homework 1 - Reinforcement Learning (2022/2023)

**Name:** GRISLAIN Clémence

## Instructions

* The deadline is **November 10 at 11:59 pm (Paris time).**

* By doing this homework you agree to the late day policy, collaboration and misconduct rules reported on [Piazza](https://piazza.com/class/l4y5ubadwj64mb/post/6).

* **Mysterious or unsupported answers will not receive full credit**. A correct answer, unsupported by calculations, explanation, or algebraic work will receive no credit; an incorrect answer supported by substantially correct calculations and explanations might still receive partial credit.

* Answers should be provided in **English**.

# Colab setup

In [ ]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
  # install rlberry library
  !pip install git+https://github.com/rlberry-py/rlberry.git@mva2021#egg=rlberry[default] > /dev/null 2>&1

  # install ffmpeg-python for saving videos
  !pip install ffmpeg-python > /dev/null 2>&1

  # packages required to show video
  !pip install pyvirtualdisplay > /dev/null 2>&1
  !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

  print("Libraries installed, please restart the runtime!")


In [ ]:
# Create directory for saving videos
!mkdir videos > /dev/null 2>&1

# Initialize display and import function to show videos
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video

In [ ]:
# Useful libraries
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

# Preparation

In the coding exercises, you will use a *grid-world* MDP, which is represented in Python using the interface provided by the [Gym](https://gym.openai.com/) library. The cells below show how to interact with this MDP and how to visualize it.


In [ ]:
from rlberry.envs import GridWorld

def get_env():
  """Creates an instance of a grid-world MDP."""
  env = GridWorld(
      nrows=5,
      ncols=7,
      reward_at = {(0, 6):1.0},
      walls=((0, 4), (1, 4), (2, 4), (3, 4)),
      success_probability=0.9,
      terminal_states=((0, 6),)
  )
  return env

def render_policy(env, policy=None, horizon=50):
  """Visualize a policy in an environment

  Args:
    env: GridWorld
        environment where to run the policy
    policy: np.array
        matrix mapping states to action (Ns).
        If None, runs random policy.
    horizon: int
        maximum number of timesteps in the environment.
  """
  env.enable_rendering()
  state = env.reset()                       # get initial state
  for timestep in range(horizon):
      if policy is None:
        action = env.action_space.sample()  # take random actions
      else:
        action = policy[state]
      next_state, reward, is_terminal, info = env.step(action)
      state = next_state
      if is_terminal:
        break
  # save video and clear buffer
  env.save_video('./videos/gw.mp4', framerate=5)
  env.clear_render_buffer()
  env.disable_rendering()
  # show video
  show_video('./videos/gw.mp4')


In [ ]:
# Create an environment and visualize it
env = get_env()
render_policy(env)  # visualize random policy

# The reward function and transition probabilities can be accessed through
# the R and P attributes:
print(f"Shape of the reward array = (S, A) = {env.R.shape}")
print(f"Shape o f the transition array = (S, A, S) = {env.P.shape}")
print(f"Reward at (s, a) = (1, 0): {env.R[1, 0]}")
print(f"Prob[s\'=2 | s=1, a=0]: {env.P[1, 0, 2]}")
print(f"Number of states and actions: {env.Ns}, {env.Na}")

# The states in the griworld correspond to (row, col) coordinates.
# The environment provides a mapping between (row, col) and the index of
# each state:
print(f"Index of state (1, 0): {env.coord2index[(1, 0)]}")
print(f"Coordinates of state 5: {env.index2coord[5]}")

# Part 1 - Dynamic Programming

## Question 1.1

Consider a general MDP with a discount factor of $\gamma < 1$. Assume that the horizon is infinite (so there is no termination). A policy $\pi$ in this MDP
induces a value function $V^\pi$. Suppose an affine transformation is applied to the reward, what is
the new value function? Is the optimal policy preserved?



### **Answer**

$V^{\pi}(s_t) = \mathbb{E}(\sum_{t=1}^{\inf} \gamma^t r(s_t, a_t) | s_0, a_t\sim\pi(s_t))$

$ \pi^* \in \underset{\pi}{\operatorname{argmax}} \left( V^{\pi} \right) $

\\
affine transformation: $r = a\times r + b, (a,b)\in \mathbb{R}^2$

$V'^{\pi}(s_t) = \mathbb{E}\left(\sum_{t=1}^{\inf} \gamma^t (a \times r(s_t, a_t) + b) | s_0, a_t\sim\pi(s_t)\right)$
$V'^{\pi}(s_t) = \mathbb{E}\left(\sum_{t=1}^{\inf} \gamma^t (a \times r(s_t, a_t)) | s_0, a_t\sim\pi(s_t)\right) + b \times \sum_{t=1}^{\inf} \gamma^t$
$V'^{\pi}(s_t) = a \times\mathbb{E}\left(\sum_{t=1}^{\inf} \gamma^t r(s_t, a_t) | s_0, a_t\sim\pi(s_t)\right) + \frac{b}{1-\gamma} $
$V'^{\pi}(s_t) = a \times V^{\pi}(s_t)+\frac{b}{1-\gamma} $


\\
*   if $a=0$:

$$\pi'^* \in \underset{\pi}{\operatorname{argmax}} \left( V'^{\pi} \right) = \underset{\pi}{\operatorname{argmax}} \left( \frac{b}{1-\gamma} \right) = \Pi $$
The optimal policy is not preserved.
*   if $a<0$:
$$\pi'^* \in \underset{\pi}{\operatorname{argmax}} \left( V'^{\pi} \right) = \underset{\pi}{\operatorname{argmax}} \left( a \times V^{\pi}(s_t) + \frac{b}{1-\gamma}\right) =  \underset{\pi}{\operatorname{argmax}} \left( a \times V^{\pi}(s_t) \right) = \underset{\pi}{\operatorname{argmin}} \left( V^{\pi} \right)$$

The optimal policy is not preserved.

*   if $a>0$:
$$\pi'^* \in \underset{\pi}{\operatorname{argmax}} \left( V'^{\pi} \right) = \underset{\pi}{\operatorname{argmax}} \left( a \times V^{\pi}(s_t) + \frac{b}{1-\gamma}\right) =  \underset{\pi}{\operatorname{argmax}} \left( a \times V^{\pi}(s_t) \right) = \underset{\pi}{\operatorname{argmax}} \left( V^{\pi} \right)$$

The optimal policy is  preserved.








## Question 1.2

Consider an infinite-horizon $\gamma$-discounted MDP. We denote by $Q^*$ the $Q$-function of the optimal policy $\pi^*$. Prove that, for any function $Q(s, a)$ (which is **not** necessarily the value function of a policy), the following inequality holds for any state $s$:

$$
V^{\pi_Q}(s) \geq V^*(s) - \frac{2}{1-\gamma}||Q^*-Q||_\infty,
$$

where $||Q^*-Q||_\infty = \max_{s, a} |Q^*(s, a) - Q(s, a)|$ and $\pi_Q(s) \in \arg\max_a Q(s, a)$. Can you use this result to show that any policy $\pi$ such that $\pi(s) \in \arg\max_a Q^*(s, a)$ is optimal?

### **Answer**
$\forall s\in S$

$V^{\pi_Q}(s) = \underset{a}{\operatorname{max}}Q^{\pi_Q}(s,a) = Q^{\pi_Q}(s, \pi_Q(s))$

$V^*(s) = Q^*(s, \pi^*(s))$

\\
Thus, $\forall s \in S$, 

$V^*(s) - V^{\pi_Q}(s)$ 

$= Q(s, \pi^*(s)) - Q^{\pi_Q}(s, \pi_Q(s))$

$= Q^*(s, \pi^*(s)) - Q^*(s, \pi_Q(s)) + \underbrace{Q^*(s, \pi_Q(s)) - Q^{\pi_Q}(s, \pi_Q(s))}_{r(s,\pi_Q(s)) + \gamma \underset{s' \in S}{\sum} p(s'|s, \pi_Q(s)) V^*(s') - r(s,\pi_Q(s)) - \gamma \underset{s' \in S}{\sum} p(s'|s, \pi_Q(s)) V^{\pi_Q}(s')}$

$= Q^*(s, \pi^*(s)) - Q^*(s, \pi_Q(s)) + \gamma \underset{s' \in S}{\sum} p(s'|s, \pi_Q(s)) \left( V^*(s') - V^{\pi_Q}(s')\right)$

$= Q^*(s, \pi^*(s)) - \underbrace{Q(s, \pi_Q(s))}_{ = \underset{a}{\operatorname{max}}Q(s,a)  \geq Q(s, \pi^*(s))} + Q(s, \pi_Q(s)) - Q^*(s, \pi_Q(s)) + \gamma \underset{s' \in S}{\sum} p(s'|s, \pi_Q(s)) \left( V^*(s') - V^{\pi_Q}(s')\right)$

$\leq Q^*(s, \pi^*(s)) - Q(s, \pi^*(s)) + Q(s, \pi_Q(s)) - Q^*(s, \pi_Q(s)) + \gamma \underset{s' \in S}{\sum} p(s'|s, \pi_Q(s)) \left( V^*(s') - V^{\pi_Q}(s')\right)$

$\leq |Q^*(s, \pi^*(s)) - Q(s, \pi^*(s))| + |Q^*(s, \pi_Q(s)) -Q(s, \pi_Q(s)) | + \gamma \underset{s' \in S}{\sum} p(s'|s, \pi_Q(s)) \left( V^*(s') - V^{\pi_Q}(s')\right)$

$\leq 2 \times ||Q^* - Q||_{\infty} + \gamma \underset{s' \in S}{\sum} p(s'|s, \pi_Q(s)) \left( V^*(s') - V^{\pi_Q}(s')\right)$

$\leq 2 \times ||Q^* - Q||_{\infty} + \gamma \times ||V^* - V^{\pi_Q}||_{\infty}$


hence we have, $\forall s \in S$

$$V^*(s) - V^{\pi_Q}(s) \leq 2 \times ||Q^* - Q||_{\infty} + \gamma \times ||V^* - V_Q||_{\infty}$$

$$ \implies ||V^* - V^{\pi_Q}||_{\infty} \leq 2 \times ||Q^* - Q||_{\infty} + \gamma \times ||V^* - V_Q||_{\infty}$$

$$ \implies ||V^* - V^{\pi_Q}||_{\infty} \leq \frac{2}{1-\gamma} \times ||Q^* - Q||_{\infty}$$

$$ \implies V^*(s) - V^{\pi_Q}(s) \leq \frac{2}{1-\gamma} \times ||Q^* - Q||_{\infty}$$

\\
Let $\pi_1$ be a policy st $\forall s \in S, \pi_1(s) \in \arg\max_a Q^*(s, a)$. According to the previous result, we have:
$$ V^{\pi_1}(s) \geq V^*(s) - \frac{2}{1-\gamma}||Q^*-Q^*||_\infty $$
$$ \implies V^{\pi_1}(s) \geq V^*(s)$$
and by definition of $V^* = \underset{\pi}{\text{max }}V^{\pi}$
$$\implies V^{\pi_1}(s) = V^*(s)$$
$$\implies \pi_1 \text{ is optimal}$$

## Question 1.3

In this question, you will implement and compare the policy and value iteration algorithms for a finite MDP. 

Complete the functions `policy_evaluation`, `policy_iteration` and `value_iteration` below.


Compare value iteration and policy iteration. Highlight pros and cons of each method.

### **Answer**

Value iteration:


*   **Pros:** for each iteration step, complexity in $O(S^2 \times A)$.
*   **Cons:** only asympotical convergence.

Policy iteration:


*   **Pros:** convergence in a finite number of iterations.
*   **Cons:** for each iteration step, full policy iteration, complexity in $O\left(S^2\times \frac{\log(\frac{1}{tol})}{\log(\frac{1}{\gamma})}\right)$ which might be expensive.



In [ ]:
def policy_evaluation(P, R, policy, gamma=0.9, tol=1e-2):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        policy: np.array
            matrix mapping states to action (Ns)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        value_function: np.array
            The value function of the given policy
    """
    Ns, Na = R.shape
    value_function = np.zeros(Ns)
    # ====================================================
    new_value_function = np.array([R[s, a] + gamma * (P[s, a, :] @ value_function) for s,a in enumerate(policy)])
    while np.linalg.norm(new_value_function - value_function) > tol:
      value_function = new_value_function
      new_value_function = np.array([R[s, a] + gamma * (P[s, a, :] @ value_function) for s,a in enumerate(policy)])
    value_function = new_value_function
    # ====================================================
    return value_function

In [ ]:
def policy_evaluation_matrix_inv(P, R, policy, gamma=0.9):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        policy: np.array
            matrix mapping states to action (Ns)
        gamma: float
            discount factor

    Return:
        value_function: np.array
            The value function of the given policy using matrix inversion
    """
    Ns, Na = R.shape
    value_function = np.zeros(Ns)
    R_pi = np.array([R[s, a] for s,a in enumerate(policy)])
    P_pi = np.array([P[s, a, :] for s,a in enumerate(policy)])
    value_function = np.linalg.inv((np.identity(Ns) - gamma * P_pi)) @ R_pi
    return value_function

In [ ]:
def policy_iteration(P, R, gamma=0.9, tol=1e-3):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        policy: np.array
            the final policy
        V: np.array
            the value function associated to the final policy
    """
    Ns, Na = R.shape
    V = np.zeros(Ns)
    policy = np.ones(Ns, dtype=int)
    # ====================================================
    Vnew = policy_evaluation(P, R, policy, gamma=gamma, tol=tol)
    while not np.array_equal(V, Vnew):
      V = Vnew
      policy = np.argmax(R + gamma * (P @ V), axis=1)
      Vnew = policy_evaluation(P, R, policy, gamma=gamma, tol=tol)
    # ====================================================
    return policy, V

In [ ]:
def value_iteration(P, R, gamma=0.9, tol=1e-3):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        Q: final Q-function (at iteration n)
        greedy_policy: greedy policy wrt Qn
        Qfs: all Q-functions generated by the algorithm (for visualization)
    """
    Ns, Na = R.shape
    Q = np.zeros((Ns, Na))
    Qfs = [Q]
    # ====================================================
    Qnew = R + gamma * (P @ np.max(Q, axis=1))
    Qfs.append(Qnew)
    while np.linalg.norm(Q - Qnew) > tol:
      Q = Qnew
      Qnew = R + gamma * (P @ np.max(Q, axis=1))
      Qfs.append(Qnew)
    Q = Qnew
    greedy_policy = np.argmax(Q, axis=1)
    # ====================================================
    return Q, greedy_policy, Qfs

### Testing your code

In [ ]:
# Parameters
tol = 1e-5
gamma = 0.99

# Environment
env = get_env()

# run value iteration to obtain Q-values
VI_Q, VI_greedypol, all_qfunctions = value_iteration(env.P, env.R, gamma=gamma, tol=tol)

# render the policy
print("[VI]Greedy policy: ")
render_policy(env, VI_greedypol)

# compute the value function of the greedy policy using matrix inversion
# ====================================================
greedy_V = policy_evaluation_matrix_inv(env.P, env.R, VI_greedypol, gamma=gamma)
# ====================================================

# show the error between the computed V-functions and the final V-function
# (that should be the optimal one, if correctly implemented)
# as a function of time
final_V = all_qfunctions[-1].max(axis=1)
norms = [ np.linalg.norm(q.max(axis=1) - final_V) for q in all_qfunctions]
plt.plot(norms)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Value iteration: convergence")

#### POLICY ITERATION ####
PI_policy, PI_V = policy_iteration(env.P, env.R, gamma=gamma, tol=tol)
print("\n[PI]final policy: ")
render_policy(env, PI_policy)

## Uncomment below to check that everything is correct
assert np.allclose(PI_policy, VI_greedypol),\
    "You should check the code, the greedy policy computed by VI is not equal to the solution of PI"

assert np.allclose(PI_V, greedy_V),\
     "Since the policies are equal, even the value function should be"

plt.show()

# Part 2 - Tabular RL

## Question 2.1

The code below collects two datasets of transitions (containing states, actions, rewards and next states) for a discrete MDP.

For each of the datasets:

1. Estimate the transitions and rewards, $\hat{P}$ and $\hat{R}$.
2. Compute the optimal value function and the optimal policy with respect to the estimated MDP (defined by $\hat{P}$ and $\hat{R}$), which we denote by $\hat{\pi}$ and $\hat{V}$.
3. Numerically compare the performance of $\hat{\pi}$ and $\pi^\star$ (the true optimal policy), and the error between $\hat{V}$ and $V^*$ (the true optimal value function).

Which of the two data collection methods do you think is better? Why?

### **Answer**

The uniform dataset is clearly better than the one generated by random policies.

In fact, from the plots below, we can see that for all size of samples, the error ($L_2$ norm and infinite norm) between the estimated and the true value function, is greater (or equal for very large dataset) for $\hat{V}_{random}$ than for $\hat{V}_{uniform}$. The same result is obtained for the error between the true and the estimated policies (error computed thanks to the $L_0$ norm).

In the random policy dataset, as the policy is deterministic, with few iteration, some states (like the goal) won't be explored. This completelly bias the estimation of $\hat{P}$ and $\hat{R}$ (especially when the goal is not found by the policy). With a very large sample, however, the stochasticity of the next state for given (s,a) ensures the exploration and thus a good estimation of $\hat{P}$ and $\hat{R}$. In the other hand, the uniform dataset maximizes the exploration of the set of states which provides better $\hat{P}$ and $\hat{R}$ for the same size of sample.

In [ ]:
def get_random_policy_dataset(env, n_samples):
  """Get a dataset following a random policy to collect data."""
  states = []
  actions = []
  rewards = []
  next_states = []
  
  state = env.reset()
  for _ in range(n_samples):
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.step(action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)
    # update state
    state = next_state
    if is_terminal:
      state = env.reset()

  dataset = (states, actions, rewards, next_states)
  return dataset

def get_uniform_dataset(env, n_samples):
  """Get a dataset by uniformly sampling states and actions."""
  states = []
  actions = []
  rewards = []
  next_states = []
  for _ in range(n_samples):
    state = env.observation_space.sample()
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.sample(state, action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)

  dataset = (states, actions, rewards, next_states)
  return dataset

def estimate_MDP(dataset):
  Phat = np.zeros((env.Ns, env.Na, env.Ns))
  Rhat = np.zeros((env.Ns, env.Na))
  N_as = np.zeros((env.Ns, env.Na))
  (states, actions, rewards, next_states) = dataset
  for (s, a, r, sp) in zip(states, actions, rewards, next_states):
    Phat[s,a,sp] += 1
    N_as[s,a] += 1
    Rhat[s,a] += r
  N_as = N_as * (N_as > 0) + np.ones(N_as.shape) * (N_as == 0)
  Phat = Phat / N_as[:,:,None]
  Rhat = Rhat / N_as

  return Phat, Rhat

# Collect two different datasets
num_samples = 500
env = get_env()
dataset_1 = get_random_policy_dataset(env, num_samples)
dataset_2 = get_uniform_dataset(env, num_samples)


In [ ]:
# Item 3: Estimate the MDP with the two datasets; compare the optimal value
# functions in the true and in the estimated MDPs

# Dataset_1
Phat_1, Rhat_1 = estimate_MDP(dataset_1)
policy_1, _ = policy_iteration(Phat_1, Rhat_1, gamma=gamma, tol=tol)
print("[PI] Estimated MDP dataset_1: ")
print()
render_policy(env, policy_1)

# Policy evaluation in the true env
value_1 = policy_evaluation_matrix_inv(env.P, env.R, policy_1, gamma=gamma)
print("Error ||V* - V_1||_inf:", np.max(PI_V - value_1))
print("Error ||V* - V_1||_2:", np.linalg.norm(PI_V - value_1))
print("Error 1 - ||pi* - pi_1||_0:", np.sum(PI_policy == policy_1))

#Dataset_2
Phat_2, Rhat_2 = estimate_MDP(dataset_2)
policy_2, value_2 = policy_iteration(Phat_2, Rhat_2, gamma=gamma, tol=tol)
print("\n[PI] Estimated MDP dataset_2: ")
render_policy(env, policy_2)

# Policy evaluation in the true env
value_2 = policy_evaluation_matrix_inv(env.P, env.R, policy_2, gamma=gamma)
print("Error ||V* - V_2||_inf:", np.max(PI_V - value_2))
print("Error ||V* - V_2||_2:", np.linalg.norm(PI_V - value_2))
print("Error 1 - ||pi* - pi_2||_0:", np.sum(PI_policy == policy_2))

In [ ]:
N = 100

X = []
# Error on the value fonction (norm L2)
Y1, Y2, yerr_1, yerr_2 = [], [], [], []
# Error on the value function (norm inf)
T1, T2, terr_1, terr_2 = [], [], [], []
# Error on the policy (norm L0)
Z1, Z2, zerr_1, zerr_2 = [], [], [], []

for n in trange(500,5001,500):
  X.append(n)
  y1, y2, t1, t2, z1, z2 = [], [], [],[], [], []
  for _ in range(N):
    dataset_1 = get_random_policy_dataset(env, n)
    Phat_1, Rhat_1 = estimate_MDP(dataset_1)
    Q, policy_1, _= value_iteration(Phat_1, Rhat_1, gamma=gamma, tol=tol)
    value_1 = np.max(Q, axis=1)
    y1.append(np.linalg.norm(PI_V - value_1))
    t1.append(np.max(np.abs(PI_V - value_1)))
    z1.append(np.sum(PI_policy == policy_1))

    dataset_2 = get_uniform_dataset(env, n)
    Phat_2, Rhat_2 = estimate_MDP(dataset_2)
    Q, policy_2, _ = value_iteration(Phat_2, Rhat_2, gamma=gamma, tol=tol)
    value_2 = np.max(Q, axis=1)
    y2.append(np.linalg.norm(PI_V - value_2))
    t2.append(np.max(np.abs(PI_V - value_2)))
    z2.append(np.sum(PI_policy == policy_2))
  
  Y1.append(np.mean(y1))
  yerr_1.append(1.96 * np.std(y1)/np.sqrt(N))
  Y2.append(np.mean(y2))
  yerr_2.append(1.96 * np.std(y1)/np.sqrt(N))
  T1.append(np.mean(t1))
  terr_1.append(1.96 * np.std(t1)/np.sqrt(N))
  T2.append(np.mean(t2))
  terr_2.append(1.96 * np.std(t2)/np.sqrt(N))
  Z1.append(np.mean(z1))
  zerr_1.append(1.96 * np.std(z1)/np.sqrt(N))
  Z2.append(np.mean(z2))
  zerr_2.append(1.96 * np.std(z2)/np.sqrt(N))

In [ ]:
fig = plt.figure(figsize=(25, 5))

fig.add_subplot(1,3,1)
plt.errorbar(X, Y1, yerr=yerr_1, label="random policy")
plt.errorbar(X, Y2, yerr=yerr_2, label="uniform")
plt.title('Error on value function (L2 norm)')
plt.legend()

fig.add_subplot(1,3,2)
plt.errorbar(X, T1, yerr=terr_1, label="random policy")
plt.errorbar(X, T2, yerr=terr_2, label="uniform")
plt.title('Error on value function (infinite norm)')
plt.legend()

fig.add_subplot(1,3,3)
plt.errorbar(X, [1 - z for z in Z1], yerr=zerr_1, label="random policy")
plt.errorbar(X, [1 - z for z in Z2], yerr=zerr_2, label="uniform")
plt.title('Error on policy (using L0 norm)')
plt.legend()


## Question 2.2

Suppose that $\hat{P}$ and $\hat{R}$ are estimated from a dataset of exactly $N$ i.i.d. samples from **each** state-action pair. This means that, for each $(s,a)$, we have $N$ samples $\{(s_1',r_1, \dots, s_N', r_N\}$, where $s_i' \sim P(\cdot | s,a)$ and $r_i \sim R(s,a)$ for $i=1,\dots,N$, and
$$ \hat{P}(s'|s,a) = \frac{1}{N}\sum_{i=1}^N \mathbb{1}(s_i' = s'), $$
$$ \hat{R}(s,a) = \frac{1}{N}\sum_{i=1}^N r_i.$$
Suppose that $R$ is a distribution with support in $[0,1]$. Let $\hat{V}$ be the optimal value function computed in the empirical MDP (i.e., the one with transitions $\hat{P}$ and rewards $\hat{R}$). For any $\delta\in(0,1)$, derive an upper bound to the error

$$ \| \hat{V} - V^* \|_\infty $$

which holds with probability at least $1-\delta$.

**Note** Your bound should only depend on deterministic quantities like $N$, $\gamma$, $\delta$, $S$, $A$. It should *not* dependent on the actual random samples.

**Hint** The following two inequalities may be helpful.

1. **A (simplified) lemma**. For any state $\bar{s}$,

$$ |\hat{V}(\bar{s}) - V^*(\bar{s})| \leq \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right|$$

2. **Hoeffding's inequality**. Let $X_1, \dots X_N$ be $N$ i.i.d. random variables bounded in the interval $[0,b]$ for some $b>0$. Let $\bar{X} = \frac{1}{N}\sum_{i=1}^N X_i$ be the empirical mean. Then, for any $\epsilon > 0$,

$$ \mathbb{P}(|\bar{X} - \mathbb{E}[\bar{X}]| > \epsilon) \leq 2e^{-\frac{2N\epsilon^2}{b^2}}.$$

### **Answer**
$\forall \bar{s} \in S$

$|\hat{V}(\bar{s}) - V^*(\bar{s})| \leq \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right|$
$\implies \| \hat{V} - V^* \|_\infty \leq \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right|$

\\
Lets define: \\
$\overline{X}(a,s) = \hat{R}(s,a) + \gamma \sum_{s'}\hat{P}(s'|s,a) V^*(s')$

$\mathbb{E}\left(\overline{X}(a,s)\right) = R(s,a) + \gamma \sum_{s'}P(s'|s,a) V^*(s')$

\\
And we have: \\
$V^*(s') = \mathbb{E}\left( \overset{\infty}{\underset{t=0}{\sum}} \gamma^t R(s_t, a_t)\right) \leq \frac{R_{max}}{1-\gamma} = \frac{1}{1-\gamma}$

$\implies$ $\forall (a,s)\in A\times S$, $ \overline{X}(a,s) = \hat{R}(s,a) + \gamma \sum_{s'}\hat{P}(s'|s,a) V^*(s') \leq 1 + \frac{\gamma}{1-\gamma} = \frac{1}{1-\gamma}$

\\
$$\mathbb{P}\left( \max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right| < \epsilon \right)$$
$$ = \mathbb{P}\left( \underset{a,s}{\bigcap} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right| < \epsilon \right)$$

by independance
$$ = \underset{a,s}{\Pi} \left[\mathbb{P}\left(\left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right| < \epsilon \right)\right]$$
$$= \underset{a,s}{\Pi} \left[\mathbb{P}\left(\left|\overline{X}(a,s) -   \mathbb{E}\left(\overline{X}(a,s)\right)\right| < \epsilon \right)\right]$$

by applying Holder inequality on each $\overline{X}(a,s)$ we obtain

$$\mathbb{P}\left( \max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right| < \epsilon \right) \geq \left(1 - 2e^{-\frac{2N\epsilon^2}{b^2}}\right)^{A\times S}$$

\\
We want $$ \left(1 - 2e^{-\frac{2N\epsilon^2}{b^2}}\right)^{A\times S} = 1-\delta \implies \epsilon = \frac{b}{\sqrt{2N}}\sqrt{log\left( \frac{1 - (1-\delta)^{SA}}{2}\right)}$$ with $b=\frac{1}{1-\gamma}$ upper bound on $\overline{X}(a,s)$ $\forall (a,s)\in A\times S$.

\\
Hence we have: \\
$$ \mathbb{P}\left( \| \hat{V} - V^* \|_\infty  <  \frac{1}{(1-\gamma)^2\sqrt{2N}}\sqrt{log\left( \frac{1 - (1-\delta)^{SA}}{2}\right)}\right)  \geq \mathbb{P}\left( \frac{1}{1-\gamma}\max_{s,a} \left| R(s,a) - \hat{R}(s,a) + \gamma \sum_{s'}(P(s'|s,a) - \hat{P}(s'|s,a)) V^*(s') \right| < \frac{\epsilon}{1 - \gamma} \right) \geq 1-\delta$$

## Question 2.3

Suppose once again that we are given a dataset of $N$ samples in the form of tuples $(s_i,a_i,s_i',r_i)$. We know that each tuple contains a valid transition from the true MDP, i.e., $s_i' \sim P(\cdot | s_i, a_i)$ and $r_i \sim R(s_i,a_i)$, while the state-action pairs $(s_i,a_i)$ from which the transition started can be arbitrary.

Suppose we want to apply Q-learning to this MDP. Can you think of a way to leverage this offline data to improve the sample-efficiency of the algorithm? What if we were using SARSA instead?

### **Answer**



1.   One way to use this dataset is to compute a good estimation of teh value function with linear fitted Q-itteration, and then to start from this estimation to apply Q-learning and from the greaddy policy of this estimated value function to start SARSA.
2.   Another way could be to use the dataset to have a better estimation of the Bellman error. In fact, for a given $(s_t, a_t, r_t, s_{t+1})$ instead of computed the Bellman error with $r_t$ we could use the mean reward found on the dataset $\bar{r}=\frac{1}{N(s,a)}\underset{(s,a,r,s')\in dataset}{\sum}r\times 1_{\{(s,a)=(s_t,a_t)\}}$. In a way, we try to estimate the reward distribution $\hat{R}$ like in the tabular Dyna-Q algorithm. For SARSA, as the Bellman error is also computed, we could do the same.



# Part 3 - RL with Function Approximation

## Question 3.1

Given a datset $(s_i, a_i, r_i, s_i')$ of (states, actions, rewards, next states), the Fitted Q-Iteration (FQI) algorithm proceeds as follows:


* We start from a $Q$ function $Q_0 \in \mathcal{F}$, where $\mathcal{F}$ is a function space;
* At every iteration $k$, we compute $Q_{k+1}$ as:

$$
Q_{k+1}\in\arg\min_{f\in\mathcal{F}} \frac{1}{2}\sum_{i=1}^N
\left(
  f(s_i, a_i) - y_i^k
\right)^2 + \lambda \Omega(f)
$$
where $y_i^k = r_i + \gamma \max_{a'}Q_k(s_i', a')$, $\Omega(f)$ is a regularization term and $\lambda > 0$ is the regularization coefficient.


Consider FQI with *linear* function approximation. That is, for a given feature map $\phi : S \rightarrow \mathbb{R}^d$, we consider a parametric family of $Q$ functions $Q_\theta(s,a) = \phi(s)^T\theta_a$ for $\theta_a\in\mathbb{R}^d$. Suppose we are applying FQI on a given dataset of $N$ tuples of the form $(s_i, a_i, r_i, s_i')$ and we are at the $k$-th iteration. Let $\theta_k \in\mathbb{R}^{d \times A}$ be our current parameter. Derive the *closed-form* update to find $\theta_{k+1}$, using $\frac{1}{2}\sum_a ||\theta_a||_2^2$ as regularization.

### **Answer**

$Q_{k+1} = Q_{\theta_{k+1}} \in\arg\min_{\theta\in\mathcal{\mathbb{R}^d}} \frac{1}{2}\sum_{i=1}^N
\left(
  \phi(s_i)^T\theta_{a_i} - y_i^k
\right)^2 +  \frac{\lambda}{2}\sum_a ||\theta_a||_2^2$

\\
$\theta^{k+1}$ is a critic point of the function:
$F : \theta  	\longrightarrow \frac{1}{2}\sum_{i=1}^N
\left(
  \phi(s_i)^T\theta_{a_i} - y_i^k
\right)^2 +  \frac{\lambda}{2}\sum_a ||\theta_a||_2^2$


Thus, $\forall a \in A,$ $\theta^{k+1}_a$ is a critic point of the function 
$F_a : \theta_a  	\longrightarrow \frac{1}{2}\sum_{i=1}^N
\left(
  \phi(s_i)^T\theta_{a_i} - y_i^k
\right)^2 +  \frac{\lambda}{2}\sum_a ||\theta_a||_2^2$

\\
Hence $\forall a \in A,$ $\frac{\partial F_a}{\partial a}(\theta^{k+1}_a)=0$

$\implies \sum_{i=1}^N
\left(
  \underbrace{\phi(s_i)^T\theta^{k+1}_{a_i}}_{\in \mathbb{R}} - y_i^k
\right)\phi(s_i)\mathbb{1}_{(a_i=a)} +  \lambda\theta^{k+1}_a = 0$

$\implies \sum_{i=1}^N
  \phi(s_i)\phi(s_i)^T\theta^{k+1}_{a_i}\mathbb{1}_{(a_i=a)} +  \lambda\theta^{k+1}_a = \sum_{i=1}^N y_i^k\phi(s_i)\mathbb{1}_{(a_i=a)}$

$\implies \left(\sum_{i=1}^N
  \phi(s_i)\phi(s_i)^T\mathbb{1}_{(a_i=a)} +  \lambda I_d\right)\theta^{k+1}_a = \sum_{i=1}^N y_i^k\phi(s_i)\mathbb{1}_{(a_i=a)}$

$\implies \theta^{k+1}_a = \left(\sum_{i=1}^N
  \phi(s_i)\phi(s_i)^T\mathbb{1}_{(a_i=a)} +  \lambda I_d\right)^{-1}\sum_{i=1}^N y_i^k\phi(s_i)\mathbb{1}_{(a_i=a)}$

## Question 3.2

The code below creates a larger gridworld (with more states than the one used in the previous questions), and defines a feature map. Implement linear FQI to this environment (in the function `linear_fqi()` below), and compare the approximated $Q$ function to the optimal $Q$ function computed with value iteration.

Can you improve the feature map in order to reduce the approximation error?

### **Answer**

With the defaults parameters, the value function map was coherent but the policy does not reach the goal. 
Different ideas have been tested in order to reduce the approximation error:

1)   Feature map:
*   Increase the number of parameters of the linear_fqi approximation (increase parameter dim)
*   Change the norm used to compute the similarity matrix. The default norm is the $L_2$, but other noms can be more relevant (norm $L_1$ of inifite).
*   Change the sigma used to compute the siminarity matrix.

2)  Training parameters:

*   The number of iteration. Too few iteration leads to poor generalization but too many to overfitting.
*   The number of samples. The greater the number the samples, the better the information on the environnement, but this increases the training time. A trade-off need to be found.
*   The parameter of the regularization $\lambda$


As a conclusion:

*  The linear model with dim=15 was to small to fit the true value function, thus we increased the dimension of the feature map up to 50 which converges in a few number of iterations $\approx 100$. (Smaller feature maps (size $\approx 35$) also converges but for higher number of iterations $\approx 1000$).

*  The number of samples was also too small to provide good fitting. We increased it up to 10 000.

*  Among the tree norms ($L_2$, $L_1$, $inf$), unsing the $L_1$ and the $infinite$ norms to compute the feature map leads to an estimated value function with high values in the areas close to the goal but on the other side of the obstacle. Trying to reach those values, the agent gets stuck behind the obstacle.This problem is less visible on the estimated value function computed with the $L_2$ norm. The agent reachs the goal. Thus, we can that say using $L2$ it provides the best estimation.

However, in comparison with the true value function, we see that our model only concentrates high value around the goal whereas in the true value function, the high values are also "diffused" on horizontal path leading to the passage between the two obstacles, and on the full area behind the obstacles.


In [ ]:
def get_large_gridworld():
  """Creates an instance of a grid-world MDP with more states."""
  walls = [(ii, 10) for ii in range(15) if (ii != 7 and ii != 8)]
  env = GridWorld(
      nrows=15,
      ncols=15,
      reward_at = {(14, 14):1.0},
      walls=tuple(walls),
      success_probability=0.9,
      terminal_states=((14, 14),)
  )
  return env

class GridWorldFeatureMap:
  """Create features for state-action pairs
  
  Args:
    dim: int
      Feature dimension
    sigma: float
      RBF kernel bandwidth
  """
  def __init__(self, env, dim=15, sigma=0.25, norm='L2'):
    self.index2coord = env.index2coord
    self.n_states = env.Ns
    self.n_actions = env.Na
    self.dim = dim
    self.sigma = sigma

    n_rows = env.nrows
    n_cols = env.ncols

    # build similarity matrix
    sim_matrix = np.zeros((self.n_states, self.n_states))
    for ii in range(self.n_states):
        row_ii, col_ii = self.index2coord[ii]
        x_ii = row_ii / n_rows
        y_ii = col_ii / n_cols
        for jj in range(self.n_states):
            row_jj, col_jj = self.index2coord[jj]
            x_jj = row_jj / n_rows
            y_jj = col_jj / n_cols
            if norm=='L2':
              dist = np.sqrt((x_jj - x_ii) ** 2.0 + (y_jj - y_ii) ** 2.0)
            elif norm=='inf':
              dist = max((x_jj - x_ii), (y_jj - y_ii))
            elif norm=='L1':
              dist = abs((x_jj - x_ii) + (y_jj - y_ii))
            else:
              assert False, "Wrong norm name"
            sim_matrix[ii, jj] = np.exp(-(dist / sigma) ** 2.0)

    # factorize similarity matrix to obtain features
    uu, ss, vh = np.linalg.svd(sim_matrix, hermitian=True)
    self.feats = vh[:dim, :]

  def map(self, observation):
    feat = self.feats[:, observation].copy()
    return feat

In [ ]:
env = get_large_gridworld()
feat_map = GridWorldFeatureMap(env)

# Visualize large gridworld
render_policy(env)

# The features have dimension (feature_dim).
feature_example = feat_map.map(1) # feature representation of s=1
print(feature_example)

# Initial vector theta representing the Q function
theta = np.zeros((feat_map.dim, env.action_space.n))
print(theta.shape)
print(feature_example @ theta) # approximation of Q(s=1, a)

In [ ]:
def linear_fqi(env, feat_map, num_iterations, lambd=0.1, gamma=0.95):
  """
  # Linear FQI implementation
  # TO BE COMPLETED
  """
  N = 10000
  dataset = get_uniform_dataset(env, n_samples=N)
  (states, actions, rewards, next_states) = dataset
  theta = np.zeros((feat_map.dim, env.Na))

  for it in trange(num_iterations):
    A = np.zeros((feat_map.dim, feat_map.dim, env.Na))
    b = np.zeros((feat_map.dim, env.Na))
    for (s, a, r, sp) in zip(states, actions, rewards, next_states):
      phi_s = feat_map.map(s).reshape((feat_map.dim, 1))
      phi_sp = feat_map.map(sp).reshape((feat_map.dim, 1))
      # print(feat.shape)
      A[:,:,a] += phi_s @ np.transpose(phi_s)
      temp =  (r + gamma * np.max(np.transpose(phi_sp) @ theta, axis=1)[0]) * phi_s
      b[:,a] += temp[:,0]
    for a in range(env.Na):
      theta[:,a] = np.linalg.solve(A[:,:,a] + lambd * np.identity(feat_map.dim), b[:,a])
  return theta

# ----------------------------
# Environment and feature map
# ----------------------------
env = get_large_gridworld()
# you can change the parameters of the feature map, and even try other maps!
norm = 'L1'
feat_map = GridWorldFeatureMap(env, dim=50, sigma=0.30, norm=norm)

# -------
# Run FQI
# -------
theta = linear_fqi(env, feat_map, num_iterations=100)

In [ ]:
# Compute and run greedy policy
Q_fqi = np.zeros((env.Ns, env.Na))
for ss in range(env.Ns):
  state_feat = feat_map.map(ss)
  Q_fqi[ss, :] = state_feat @ theta

V_fqi = Q_fqi.max(axis=1)
policy_fqi = Q_fqi.argmax(axis=1)
render_policy(env, policy_fqi, horizon=100)

# Visualize the approximate value function in the gridworld.
img = env.get_layout_img(V_fqi)    
plt.imshow(img)
plt.title("Estimated value function (feature map with norm {})".format(norm))
plt.show()

In [ ]:
# Compute and run greedy policy
Q_fqi = np.zeros((env.Ns, env.Na))
for ss in range(env.Ns):
  state_feat = feat_map.map(ss)
  Q_fqi[ss, :] = state_feat @ theta

V_fqi = Q_fqi.max(axis=1)
policy_fqi = Q_fqi.argmax(axis=1)
render_policy(env, policy_fqi, horizon=100)

# Visualize the approximate value function in the gridworld.
img = env.get_layout_img(V_fqi)    
plt.imshow(img)
plt.title("Estimated value function (feature map with norm {})".format(norm))
plt.show()

In [ ]:
# Compute and run greedy policy
Q_fqi = np.zeros((env.Ns, env.Na))
for ss in range(env.Ns):
  state_feat = feat_map.map(ss)
  Q_fqi[ss, :] = state_feat @ theta

V_fqi = Q_fqi.max(axis=1)
policy_fqi = Q_fqi.argmax(axis=1)
render_policy(env, policy_fqi, horizon=100)

# Visualize the approximate value function in the gridworld.
img = env.get_layout_img(V_fqi)    
plt.imshow(img)
plt.title("Estimated value function (feature map with norm {})".format(norm))
plt.show()

In [ ]:
# True value function
tol = 1e-5
gamma=0.95
true_Q, _, _ = value_iteration(env.P, env.R, gamma=gamma, tol=tol)
true_V = true_Q.max(axis=1)

# Visualize the true value function in the gridworld.
img = env.get_layout_img(true_V)    
plt.imshow(img)
plt.title("True value function")
plt.show()